In [1]:
#@title Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic intersect"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

import scipy.integrate
import bokeh.io
import numpy as np
from intersect import intersection

import biocircuits
import numba

from skimage import measure
from intersect import intersection
import sympy as sp
import pylab as pl

import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import Range1d
from bokeh.io import export_svgs

bokeh.io.output_notebook()

In [2]:
# Some auxiliary functions

def JoinMatrices(A,B):

  # Function to create two matrices of differense size by placing them in a diagonal [A, 0 ; 0, B]

  n1, m1 = A.shape
  n2, m2 = B.shape
  ZA = np.zeros((n1,m2))
  ZB = np.zeros((n2,m1))
  tmp1 = np.hstack((A,ZA))
  tmp2 = np.hstack((ZB,B))
  return np.vstack((tmp1,tmp2))

In [3]:
#@title Negative feedback functions

# Ordinary differential equations
def IFFL_NF(X, t, a, K, m, d, b, k, xi, th, g):
    """
    Right hand desribe production and degradation and return dx/dt
    """
    y1, y2, u1, u2, x = X

    return (
      [
          a * K**m / (K**m + y2**m) - d*y1 + b*u1, # d y1 / dt =
          a * K**m / (K**m + y1**m) - d*y2  , # d y1 / dt = # u2 stabilize
          k * x - d*u1 - g*u1*u2, # d u2 / dt =
          xi * K**m / (K**m + y1**m) - d*u2 - g*u1*u2, # d u2 / dt =
          th * K**m / (K**m + y1**m) - d*x, # d u2 / dt =
      ]
  )

# Nulcliness analysis
def Compute_IO(y, a, K, m, d, b, k, xi, th, g):
  x = th/d*K**m/(K**m + y**m)
  f1 = k*x
  f2 = xi*K**m/(K**m + y**m)

  A = (f2-f1)/d + d/g
  B = -f1/g

  u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d
  y2_2 = a/d*K**m/(K**m + y**m)

  return (y2_1,y2_2, u1)

def Compute_IO_2(y, a, K, m, d, b, k, xi, th, g):
    x = th/d*K**m/(K**m + y**m)
    f1 = k*x
    f2 = xi*K**m/(K**m + y**m)

    A = (f2-f1)/d + d/g
    B = -f1/g

    u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

    ym, tmp = intersection(y, a + b*u1, y, d*y)
    #print(ym)

    y1_1 = np.linspace(0,ym[0]*1.2,len(y))

    y2_2 = a/d*K**m/(K**m + y**m)

    return (y2_2, y1_1)

def Compute_IO_1(y, a, K, m, d, b, k, xi, th, g):
    x = th/d*K**m/(K**m + y**m)
    f1 = k*x
    f2 = xi*K**m/(K**m + y**m)

    A = (f2-f1)/d + d/g
    B = -f1/g

    u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

    y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
    return (y2_1)

# Simple propensity function for Gillespie algorithm (stochastic)
def simple_propensity_NF(propensities, population, t, a, K, m, d, b, k, xi, th, g, Omega):
    y1, y2, u1, u2, x = population

    # Toggle switch's equations
    propensities[0] = a * Omega * (K*Omega)**m/((K*Omega)**m + y2**m) # 0 -> Y1
    propensities[1] = d * y1 # Y1 -> 0
    propensities[2] = a * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # 0 -> Y1
    propensities[3] = d * y2 # Y2 -> 0
    propensities[4] = b * u1 # 0 -> Y1

    # Adaptive controller's equations
    propensities[5] = k * x # X -> X + U1
    propensities[6] = d * u1 # U1 -> 0
    propensities[7] = xi * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # Y1 -> Y1 + U2
    propensities[8] = d * u2 # U2 -> 0
    propensities[9] = th * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # Y1 -> Y1 + X
    propensities[10] = d * x # X -> 0
    propensities[11] = g * u1 * u2 / Omega # U1 + U2 -> 0

# Adaptive controller with a positive feedback architecture

If we consider the case were the feedback comes from the non-target specie ($Y_2$), we will be dealing with a positive feedback architecture. In the following simulations, we will consider the same metrics defined for the negative feedback architecture to analyze this new control strategy. The new system´s ODEs remain the same

\begin{eqnarray}
  \frac{dy_1}{dt} &=& \alpha \frac{K^m}{K^m + y_2^m} - \delta y_1 + \beta u_1 \\
  \frac{dy_2}{dt} &=& \alpha \frac{K^m}{K^m + y_1^m} - \delta y_2 \\
\end{eqnarray}

The equations that describe the controller are the ones that should be modified as follows:

\begin{eqnarray}
  \frac{du_1}{dt} = kx - \delta u_1 - \gamma u_1 u_2, \space \frac{du_2}{dt} = \xi \frac{K^m}{K^m + y_2^m} - \delta u_2 - \gamma u_1 u_2, \space  \frac{dx}{dt} = \theta \frac{K^m}{K^m + y_2^m} - \delta x
\end{eqnarray}

## Finding the steady state
\begin{eqnarray}
\bar y_2 &=& \frac{\alpha}{\delta} \left (\frac{K^m}{K^m + \bar y_1^m} \right ) \\
\bar y_1 &=& \frac{\alpha}{\delta}\frac{K^m}{K^m + \bar{y_2}^m} + \frac{\beta}{\delta}u_1
\end{eqnarray}

As opposite to the negative feedback case, we should now find $\bar u_1$ as a function of $\bar y_2$,
\begin{eqnarray}
\bar u_2 &= \frac{f_1 - \delta \bar u_1}{\gamma \bar u_1} = \frac{f_2}{\gamma \bar u_1 + \delta},
\end{eqnarray}
where $f_1 = k \bar x$, $f_2 = \xi \frac{K^m}{K^m + \bar y_2^m}$, and $\bar x = \frac{\theta}{\delta} \frac{K^m}{K^m + \bar y_2^m}$.

This leads also to a second order polynomial, $P(\bar u_1) = \bar u_1^2 + A \bar u_1 + B = 0$, and leads to a single positive real solution
\begin{eqnarray}
\bar u_1 &= \frac{-A + \sqrt{A^2 - 4B}}{2}
\end{eqnarray}
where $A = \frac{f_2-f_1}{\delta} + \frac{\delta}{\gamma}$, and $B = - \frac{f_1}{\gamma}$.




In [5]:
#@title Positive feedback architecture

# Ordinary Differential Equations (ODEs)
def IFFL_PF(X, t, a, K, m, d, b, k, xi, th, g):
    """
    Right hand desribe production and degradation and return dx/dt
    """
    y1, y2, u1, u2, x = X

    return (
      [
          a * K**m / (K**m + y2**m) - d*y1 + b*u1, # d y1 / dt =
          a * K**m / (K**m + y1**m) - d*y2  , # d y1 / dt = # u2 stabilize
          k * x - d*u1 - g*u1*u2, # d u2 / dt =
          xi * K**m / (K**m + y2**m) - d*u2 - g*u1*u2, # d u2 / dt =
          th * K**m / (K**m + y2**m) - d*x, # d u2 / dt =
      ]
  )

# Nulcliness equations
def PF_null(y1_1, y2_2, a, K, m, d, b, k, xi, th, g):
  y2_1 = a/d * K**m / (K**m + y1_1**m)

  x = th/d * K**m / (K**m + y2_2**m)
  f1 = k*x
  f2 = xi * K**m / (K**m + y2_2**m)

  A = (f2-f1)/d + d/g
  B = -f1/g

  u1 = 0.5*(-A + (A**2 - 4*B)**0.5)

  y1_2 = (a/d) * K**m / (K**m + y2_2**m) + (b/d) * u1

  return (y2_1, y1_2)

def simple_propensity_PF(propensities, population, t, a, K, m, d, b, k, xi, th, g, Omega):
    y1, y2, u1, u2, x = population

    # Toggle switch's equations
    propensities[0] = a * Omega * (K*Omega)**m/((K*Omega)**m + y2**m) # 0 -> Y1
    propensities[1] = d * y1 # Y1 -> 0
    propensities[2] = a * Omega * (K*Omega)**m/((K*Omega)**m + y1**m) # 0 -> Y1
    propensities[3] = d * y2 # Y2 -> 0
    propensities[4] = b * u1 # 0 -> Y1

    # Adaptive controller's equations
    propensities[5] = k * x # X -> X + U1
    propensities[6] = d * u1 # U1 -> 0
    propensities[7] = xi * Omega * (K*Omega)**m/((K*Omega)**m + y2**m) # Y1 -> Y1 + U2
    propensities[8] = d * u2 # U2 -> 0
    propensities[9] = th * Omega * (K*Omega)**m/((K*Omega)**m + y2**m) # Y1 -> Y1 + X
    propensities[10] = d * x # X -> 0
    propensities[11] = g * u1 * u2 / Omega # U1 + U2 -> 0

# Stoichiometry  matrix of the toggle switch
# Y1, Y2
S1 = np.array(
    [
     [ 1, 0],  # 0 -> Y1
     [-1, 0],  # Y1 -> 0
     [ 0, 1],  # 0 -> Y2
     [ 0,-1],  # Y2 -> 0
     [ 1, 0],  # 0 -> Y1
    ],
    dtype=int,
)
#print(S1)

# Stoichiometry  matrix of the adaptive controller
# U1, U2, X
S2 = np.array(
    [
     [ 1, 0, 0], # X -> U1
     [-1, 0, 0], # U1 -> 0
     [ 0, 1, 0], # Y1 -> U2
     [ 0,-1, 0], # U2 -> 0
     [ 0, 0, 1], # Y1 -> X
     [ 0, 0,-1], # X -> 0
     [-1,-1, 0], # U1 + u2 -> 0
    ],
    dtype=int,
)

# Stoichiometry  matrix of the closed-loop system
simple_update = JoinMatrices(S1,S2)

## Comparison between the adaptive controller with a negative feedback and positive feedback architecture

In [ ]:
#@title Dynamics of the closed-loop system

a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 100 # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 5 # gain [1/h]

# Time interval
t = np.linspace(0,20,400)

# Set up plots
fig_size = [320, 225] # to visualize

plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          #x_range=x_range,y_range=y_range
                          x_axis_label = 'time [h]', y_axis_label = 'y [uM]'),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Color palette
palette = bokeh.palettes.grey(9)[::-1]

# Initial conditions favoring y2
vecA, vecB = np.linspace(1, 3, 7), np.ones(7) * 2.9
vec_y1, vec_y2 = np.concatenate([vecB, vecA]), np.concatenate([vecA, vecB])

# Set arguments for each scenario
scenarios = [
    (IFFL_PF, (a, K, m, d, b * 0, k, xi, th, g), plots[0], 'without controller'),
    (IFFL_NF, (a, K, m, d, b, k, xi, th, g), plots[1], 'IFFL - Negative feedback'),
    (IFFL_PF, (a, K, m, d, b, k, xi, th, g), plots[2], 'IFFL - Positive feedback')
]

# Loop through scenarios
for ode_func, args, plot, title in scenarios:
    for ii in range(len(vec_y1)):
        x01 = np.array([vec_y1[ii], vec_y2[ii], 0., 0., 0.])
        x1 = scipy.integrate.odeint(ode_func, x01, t, args=args).transpose()

        # Plotting
        plot.line(t, x1[0, :], line_width=2, color="black", legend_label='y1')
        plot.line(t, x1[1, :], line_width=2, color="orange", legend_label='y2')

    plot.title.text = title
    plot.legend.background_fill_alpha = 0.00

# Tidy up and set export settings
# for i in range(3):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
#@title The positive feedback generates a biased cell fate, too
a = 2.2 # 2.2 alpha  uM/h
K = 1 # Kappa uM
m = 3 # 3 cooperatividad
d = 1 # delta 1/h
k = 1
g = 100
th = d
xi = 1
b = 1

Omega = 100 # 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args1 = (a, K, m, d, b*0, k, xi, th, g, Omega)
args2 = (a, K, m, d, b*3, k, xi, th, g, Omega)

# Solving with Gillespie algorithm
samples1 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0,time_points, size=250, args=args1, n_threads=4,progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0,time_points, size=250, args=args2, n_threads=4,progress_bar=False)
samples3 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0,time_points, size=250, args=args2, n_threads=4,progress_bar=False)

# Set up plots
fig_size = [320, 225] # to visualize
x_range = (-300, 300)
y_range = (0, 0.025)

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range,
                          x_axis_label = 'y1 - y2 [# molecules]', y_axis_label = 'p'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args1 = (a, K, m, d, b*0, k, xi, th, g, Omega)
args2 = (a, K, m, d, b*3, k, xi, th, g, Omega)

# Solving with Gillespie algorithm
samples1 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0, time_points, size=250, args=args1, n_threads=4, progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0, time_points, size=250, args=args2, n_threads=4, progress_bar=False)
samples3 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args2, n_threads=4, progress_bar=False)

fig_size = [320, 225] # to visualize
x_range = (-300, 300)
y_range = (0, 0.025)

# Set up plots
plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range,
                          x_axis_label='y1 - y2', y_axis_label='p'))
    plots[i].axis.major_label_text_font_size = "10pt"

# Reference plot
bin0 = np.arange(-3*Omega, 3*Omega+6, 8) # 8
hist, bin_edges = np.histogram(samples1[:, -1, 0] - samples1[:, -1, 1], bin0, density=True)

# Plots setup
for plot in [plots[0], plots[2], plots[5]]:
    plot.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="grey", line_color="grey", fill_alpha=0.2, line_alpha=0)
    plot.step(bin_edges[:-1], hist, line_width=1, mode="after", color="grey", legend_label='β = 0')
plots[0].title.text = 'without controller'
plots[2].title.text = 'Overlay IFFL-NF and reference'
plots[5].title.text = 'Overlay IFFL-PF and reference'

# IFFL - Negative feedback
hist, bin_edges = np.histogram(samples2[:, -1, 0] - samples2[:, -1, 1], bin0, density=True)
for plot in [plots[1], plots[2], plots[3]]:
    plot.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="orange", line_color="orange", fill_alpha=0.2, line_alpha=0)
    plot.step(bin_edges[:-1], hist, line_width=1, mode="after", color="orange", legend_label='NF')
plots[1].title.text = 'IFFL - Negative feedback'
plots[3].title.text = 'Overlay PF and NF'

# IFFL - Positive feedback
hist, bin_edges = np.histogram(samples3[:, -1, 0] - samples3[:, -1, 1], bin0, density=True)
for plot in [plots[4], plots[5], plots[3]]:
    plot.quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="purple", line_color="purple", fill_alpha=0.2, line_alpha=0)
    plot.step(bin_edges[:-1], hist, line_width=1, mode="after", color="purple", legend_label='PF')
plots[4].title.text = 'IFFL - Positive feedback'

# Tidy up and set export settings
for i in range(6):
  plots[i].legend.background_fill_alpha = 0.0
  plots[i].legend.location = 'top'
  #plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.gridplot([[plots[0], plots[1], plots[2]], [plots[3], plots[4], plots[5]]]))

In [ ]:
#@title Phase plane visualization

# IMPORTANT: run the previous code chunk ("The positive feedback generates a biased cell fate, too") before running the present chunk

# Set up plots
fig_size = [300, 225] # to visualize
x_range = (0, 3.1)
y_range = (0, 3.1)

plots = []
for i in range(3):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range,
                          x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Definition of the state variable
y = np.linspace(0,3,100)

# for the system without adaptive controller
y2_1, y2_2, u1 = Compute_IO(y, a, K, m, d, b*0, k, xi, th, g)
plots[0].line(y,y2_1,line_width=2, color="black", ) #
plots[0].line(y,y2_2,line_width=2, color="black", ) #

# for the system with adaptive controller with negative feedback architecture
y2_1, y2_2, u1 = Compute_IO(y, a, K, m, d, b, k, xi, th, g)
plots[1].line(y,y2_1,line_width=2, color="black", ) #
plots[1].line(y,y2_2,line_width=2, color="black", ) #

# for the system with adaptive controller with positive feedback architecture
y2_1, y1_2 = PF_null(y, y, a, K, m, d, b, k, xi, th, g)
plots[2].line(y,y1_2,line_width=2, color="black", ) #
plots[2].line(y2_1,y,line_width=2, color="black", ) #

# Plotting the trajectories
for i in range(100):
    sol = samples1[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        plots[0].line(sol[:,0],sol[:,1],line_width=1, color="blue",alpha= 0.1, line_join="bevel") #
    elif ratio < 1:
        plots[0].line(sol[:,0],sol[:,1],line_width=1, color="red",alpha= 0.1, line_join="bevel") #
    else:
        plots[0].line(sol[:,0],sol[:,1],line_width=1, color="black",alpha= 0.1, line_join="bevel") #

    sol = samples2[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        plots[1].line(sol[:,0],sol[:,1],line_width=1, color="blue",alpha= 0.1, line_join="bevel") #
    elif ratio < 1:
        plots[1].line(sol[:,0],sol[:,1],line_width=1, color="red",alpha= 0.1, line_join="bevel") #
    else:
        plots[1].line(sol[:,0],sol[:,1],line_width=1, color="black",alpha= 0.1, line_join="bevel") #

    sol = samples3[i,:,:]/Omega
    ratio = sol[-1,0]/sol[-1,1]
    if ratio > 1:
        plots[2].line(sol[:,0],sol[:,1],line_width=1, color="blue",alpha= 0.1, line_join="bevel") #
    elif ratio < 1:
        plots[2].line(sol[:,0],sol[:,1],line_width=1, color="red",alpha= 0.1, line_join="bevel") #
    else:
        plots[2].line(sol[:,0],sol[:,1],line_width=1, color="black",alpha= 0.1, line_join="bevel") #

# Tidy up and set export settings
plots[0].title.text = 'without controller'
plots[1].title.text = 'IFFL - Negative Feedback'
plots[2].title.text = 'IFFL - Positive Feedback'
# for i in range(3):
#   plots[i].output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-3-0b44bbe636e5>:31: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d
<ipython-input-3-0b44bbe636e5>:31: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d


## Steady state analysis

In [ ]:
#@title Preliminary steady state comparison
a = 2.2 # production rate [uM/h]
K = 1 # Kappa [uM]
m = 3 # 3 cooperativity
d = 1 # delta [1/h]
k = 1 # production rate [1/h]
g = 100 # sequestration rate [1/h/uM]
th = d # production rate [uM/h]
xi = 1 # production rate [uM/h]
b = 2 # gain [1/h]

# Time interval
t = np.linspace(0,20,400)

args1 = (a, K, m, d, b, k, xi, th, g)

fig_size = [300, 225] # to visualize
x_range = Range1d(0, 3.1)
y_range = Range1d(0, 3.1)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range,y_range=y_range,
                          x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]'
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"     #background_fill_color="#fafafa"

# Hypothesis: NF -> y1 stable & PF -> y2 stable
y1_2 = np.logspace(-5,1,500)

# Reference plot for NF
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b*0, k, xi, th, g)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b*0, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[0].line(y1_1,y2_1,line_width=2, color="black", line_dash="dashed", legend_label = 'w/o IFFL') #
plots[0].line(y1_2,y2_2,line_width=2, color="black", line_dash="dashed") #
plots[0].scatter(y1_i,y2_i, size = 6, color = 'red', fill_color = 'white', line_width = 2)

# Starting with NF with β = 2
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, k, xi, th, g)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[0].line(y1_1,y2_1,line_width=2, color="black", legend_label = 'w IFFL') #
plots[0].line(y1_2,y2_2,line_width=2, color="black",) #
plots[0].scatter(y1_i,y2_i, size = 6, color = 'blue', fill_color = 'white', line_width = 2)

plots[0].title.text = 'IFFL - Negative feedback'
plots[0].legend.background_fill_alpha = 0.0

# Reference plot for PF
y1_1 = np.logspace(-5,1,500)
y2_2 = np.logspace(-5,1,500)
y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b*0, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
plots[1].line(y1_1,y2_1,line_width=2, color="black", line_dash = 'dashed', ) #
plots[1].line(y1_2,y2_2,line_width=2, color="black", line_dash = 'dashed', legend_label = 'w/o IFFL') #
plots[1].scatter(y1_i,y2_i, size = 6, color = 'red', fill_color = 'white', line_width = 2)

# Comparing with PF with β = 2
y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
plots[1].line(y1_1,y2_1,line_width=2, color="black", ) #
plots[1].line(y1_2,y2_2,line_width=2, color="black",legend_label = 'w IFFL') #
plots[1].scatter(y1_i,y2_i, size = 6, color = 'blue', fill_color = 'white', line_width = 2)

plots[1].title.text = 'IFFL - Positive feedback'
plots[1].legend.background_fill_alpha = 0.0

bokeh.io.show(bokeh.layouts.row(plots))

<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title How does each architecture alter the equilibrium landscape?
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
gv = (10, 100, 1000)  # sequestration rates [1/h/uM]
th = d  # production rate [uM/h]
xi = 1  # production rate [uM/h]
b = 2   # gain [1/h]

vec = np.linspace(0, 5, 7)
palette = bokeh.palettes.grey(5)[::-1]  # Corrected palette name

# Set up plots
fig_size = [300, 225]  # to visualize

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_axis_label='β'))
    plots[i].axis.major_label_text_font_size = "10pt"

# Negative feedback
y1_2 = np.linspace(0, 3, 300)

for kk, gi in enumerate(gv):
    out1 = np.empty((3, len(vec))) * np.nan
    out2 = np.empty((3, len(vec))) * np.nan
    for i, bi in enumerate(vec):
        y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, bi, k, xi, th, gi)
        y2_1 = Compute_IO_1(y1_1, a, K, m, d, bi, k, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i) > 0:
            out1[0, i] = np.max(y1_i)
            out2[0, i] = np.max(y2_i)
            if len(y1_i) == 1:
                out1[2, i] = np.max(y1_i)
                out2[2, i] = np.max(y2_i)
            else:
                out1[1, i] = np.max(y1_i)
                out2[1, i] = np.max(y2_i)

    plots[0].line(vec, out1[0, :], line_width=2, color=palette[kk + 1])
    plots[0].scatter(vec, out1[1, :], line_width=1, size=7, color=palette[kk + 1], legend_label=f'{gi / g} γ ')
    plots[0].scatter(vec, out1[2, :], line_width=1, size=7, color=palette[kk + 1], fill_color='white')

    plots[1].line(vec, out2[0, :], line_width=2, color=palette[kk + 1])
    plots[1].scatter(vec, out2[1, :], line_width=1, size=7, color=palette[kk + 1])
    plots[1].scatter(vec, out2[2, :], line_width=1, size=7, color=palette[kk + 1], fill_color='white')

plots[0].yaxis.axis_label = 'y1'
plots[1].yaxis.axis_label = 'y2'
plots[0].title.text = 'Negative feedback'
plots[0].x_range = Range1d(0, 5.1)
plots[1].x_range = Range1d(0, 5.1)

# Positive feedback
y1_1 = np.linspace(0, 3, 300)
y2_2 = np.linspace(0, 3, 300)

for kk, gi in enumerate(gv):
    out1 = np.empty((3, len(vec))) * np.nan
    out2 = np.empty((3, len(vec))) * np.nan
    for i, bi in enumerate(vec):
        y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, bi, k, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i) > 0:
            out1[0, i] = np.max(y1_i)
            out2[0, i] = np.max(y2_i)
            if len(y1_i) == 1:
                out1[2, i] = np.max(y1_i)
                out2[2, i] = np.max(y2_i)
            else:
                out1[1, i] = np.max(y1_i)
                out2[1, i] = np.max(y2_i)

    plots[2].line(vec, out1[0, :], line_width=2, color=palette[kk + 1])
    plots[2].scatter(vec, out1[1, :], line_width=1, size=7, color=palette[kk + 1], legend_label=f'{gi / g} γ ')
    plots[2].scatter(vec, out1[2, :], line_width=1, size=7, color=palette[kk + 1], fill_color='white')

    plots[3].line(vec, out2[0, :], line_width=2, color=palette[kk + 1])
    plots[3].scatter(vec, out2[1, :], line_width=1, size=7, color=palette[kk + 1])
    plots[3].scatter(vec, out2[2, :], line_width=1, size=7, color=palette[kk + 1], fill_color='white')

plots[2].yaxis.axis_label = 'y1'
plots[3].yaxis.axis_label = 'y2'
plots[2].title.text = 'Positive feedback'
plots[2].x_range = Range1d(0, 5.1)
plots[3].x_range = Range1d(0, 5.1)

# Reference plot
y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b * 0, k, xi, th, gi)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b * 0, k, xi, th, gi)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
hline = bokeh.models.Span(location=np.max(y1_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[0].renderers.extend([hline])
hline = bokeh.models.Span(location=np.max(y2_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[1].renderers.extend([hline])

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b * 0, k, xi, th, gi)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)
hline = bokeh.models.Span(location=np.max(y1_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[2].renderers.extend([hline])
hline = bokeh.models.Span(location=np.max(y2_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[3].renderers.extend([hline])

# Last details for plots
plots[0].y_range = Range1d(2.1, 2.51)
plots[1].y_range = Range1d(0, 2.5)
plots[2].y_range = Range1d(0.51, 3.2)
plots[3].y_range = Range1d(0, 2.5)

plots[0].legend.location = 'top_left'
plots[0].legend.background_fill_alpha = 0.0
plots[2].legend.location = 'bottom_left'
plots[2].legend.background_fill_alpha = 0.0

bokeh.io.show(bokeh.layouts.gridplot([[plots[0], plots[1]], [plots[2], plots[3]]]))


<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


In [ ]:
#@title The negative and the positive feedback exhibit different dynamic properties above certain control gain ($\beta$) value
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
g = 100  # sequestration rates [1/h/uM]
th = d   # production rate [uM/h]
xi = 1   # production rate [uM/h]
b = 4    # gain [1/h]
Omega = 100  # 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args1 = (a, K, m, d, b * 0, k, xi, th, g, Omega)
args2 = (a, K, m, d, b, k, xi, th, g, Omega)

# Solving with Gillespie algorithm
samples1 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0, time_points, size=250, args=args1, n_threads=4, progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity_NF, simple_update, population_0, time_points, size=250, args=args2, n_threads=4, progress_bar=False)
samples3 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args2, n_threads=4, progress_bar=False)

# Set up plots for probability analysis
fig_size = [300, 225]  # to visualize

row0 = [bokeh.plotting.figure(width=fig_size[0], height=fig_size[1], x_range=Range1d(-300,300), y_range=Range1d(0, 0.02), x_axis_label = 'y1 - y2 [# molecules]', y_axis_label = 'p') for _ in range(3)]
for fig in row0:
    fig.axis.major_label_text_font_size = "10pt"

# Set up plots for nullcline analysis
row1 = [bokeh.plotting.figure(width=fig_size[0], height=fig_size[1], x_range=Range1d(0, 3.1), y_range=Range1d(0, 3.1), x_axis_label = 'y1 [uM]', y_axis_label = 'y2 [uM]') for _ in range(3)]
for fig in row1:
    fig.axis.major_label_text_font_size = "10pt"

# β = 0 as a reference
y1_2 = np.logspace(-5, 1, 500)

bin0 = np.arange(-3 * Omega, 3 * Omega + 6, 8)  # 8
hist, bin_edges = np.histogram(samples1[:, -1, 0] - samples1[:, -1, 1], bin0, density=True)

row0[0].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="grey", line_color="grey", fill_alpha=0.2, line_alpha=0.)
row0[0].step(bin_edges[:-1], hist, line_width=1, mode="after", color="grey")
row0[0].title.text = f'β = 0'

y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b * 0, k, xi, th, g)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b * 0, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

row1[0].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
row1[0].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
row1[0].scatter(y1_i, y2_i, line_width=1, color='grey', size=7)

# Evaluating the case where β = 4 (equilibrium points are lost)
hist, bin_edges = np.histogram(samples2[:, -1, 0] - samples2[:, -1, 1], bin0, density=True)

row0[1].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="orange", line_color="orange", fill_alpha=0.2, line_alpha=0.)
row0[1].step(bin_edges[:-1], hist, line_width=1, mode="after", color="orange")
row0[1].title.text = f'Negative feedback with β = 4'

y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, k, xi, th, g)
y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

row1[1].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
row1[1].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
row1[1].scatter(y1_i, y2_i, line_width=1, color='orange', size=7)

# Evaluating the same case where β = 4 but for positive feedback
y1_1 = np.logspace(-5, 1, 500)
y2_2 = np.logspace(-5, 1, 500)

hist, bin_edges = np.histogram(samples3[:, -1, 0] - samples3[:, -1, 1], bin0, density=True)

row0[2].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="purple", line_color="purple", fill_alpha=0.2, line_alpha=0.)
row0[2].step(bin_edges[:-1], hist, line_width=1, mode="after", color="purple")
row0[2].title.text = f'Positive feedback with β = 4'

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, xi, th, g)
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

row1[2].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
row1[2].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
row1[2].scatter(y1_i, y2_i, line_width=1, color='purple', size=7)

# Visualization
bokeh.io.show(bokeh.layouts.gridplot([[row0[0], row0[1], row0[2]], [row1[0], row1[1], row1[2]]]))

<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)


## Optimal parameter characterization

We can approximate the dynamics of the controller in the fast sequestration regime ($\gamma \to \infty$), and find that
\begin{eqnarray}
u_1(t) & = \max \left ( 0, L^{-1} \left ( \frac{\theta k}{(s + \delta)^2}  \left (1 - \frac{\delta \xi}{k \theta} - \frac{ \xi}{k \theta} s \right) Y^*_1(s)\right )\right )
\end{eqnarray}

In [ ]:
#@title How much can we deviate from the adaptive metric? - Part 1
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
gv = (10, 100, 1000)  # sequestration rates [1/h/uM]
th = d   # production rate [uM/h]
xi = 1   # production rate [uM/h]
b = 2    # gain [1/h]

k_vec = np.linspace(0.5, 2, 7)
vec = d * xi / th / k_vec

y1_1 = np.linspace(0, 3, 300)
y2_2 = np.linspace(0, 3, 300)

palette = bokeh.palettes.grey(5)[::-1]

# Set up plots
fig_size = [320, 225]  # to visualize
x_range = (0.4, 2.1)

plots = []
for i in range(4):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=x_range, x_axis_label = 'r'))
    plots[i].axis.major_label_text_font_size = "10pt"

# Definition of the state variable
y1_2 = np.linspace(0, 3, 300)

for kk, gi in enumerate(gv):
    out1 = np.empty((3, len(vec))) * np.nan
    out2 = np.empty((3, len(vec))) * np.nan
    for i, ki in enumerate(k_vec):
        y2_2, y1_1 = Compute_IO_2(y1_2, a, K, m, d, b, ki, xi, th, gi)
        y2_1 = Compute_IO_1(y1_1, a, K, m, d, b, ki, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i) > 0:
            out1[0, i] = np.max(y1_i)
            out2[0, i] = np.max(y2_i)
            if len(y1_i) == 1:
                out1[2, i] = np.max(y1_i)
                out2[2, i] = np.max(y2_i)
            else:
                out1[1, i] = np.max(y1_i)
                out2[1, i] = np.max(y2_i)

    plots[0].line(vec, out1[0, :], line_width=2, color=palette[kk + 1])
    plots[0].scatter(vec, out1[1, :], line_width=1, size=6, color=palette[kk + 1], legend_label=f'{gi/g} γ')
    plots[0].scatter(vec, out1[2, :], line_width=1, size=6, color=palette[kk + 1], fill_color='white')

    plots[1].line(vec, out2[0, :], line_width=2, color=palette[kk + 1])
    plots[1].scatter(vec, out2[1, :], line_width=1, size=6, color=palette[kk + 1])
    plots[1].scatter(vec, out2[2, :], line_width=1, size=6, color=palette[kk + 1], fill_color='white')

# Reference plot
y = np.linspace(0, 3, 300)
y2_1, y2_2, u1 = Compute_IO(y, a, K, m, d, b * 0, ki, xi, th, gi)
y1_i, y2_i = intersection(y, y2_1, y, y2_2)
hline = bokeh.models.Span(location=np.max(y1_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[0].renderers.extend([hline])
hline = bokeh.models.Span(location=np.max(y2_i), dimension='width', line_color='grey', line_width=1, line_dash='dashed')
plots[1].renderers.extend([hline])

# Last details for plots
#plots[0].y_range = Range1d(2.0, 2.51)
#plots[1].y_range = Range1d(0, 2.5)
plots[0].yaxis.axis_label = 'y1 [uM]'
plots[1].yaxis.axis_label = 'y2 [uM]'
plots[0].legend.background_fill_alpha = 0.0
plots[0].legend.location = 'top_right'
plots[0].title.text = 'Negative feedback'

for kk, gi in enumerate(gv):
    out1 = np.empty((3, len(vec))) * np.nan
    out2 = np.empty((3, len(vec))) * np.nan
    for i, ki in enumerate(k_vec):
        y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, ki, xi, th, gi)
        y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

        if len(y1_i) > 0:
            out1[0, i] = np.max(y1_i)
            out2[0, i] = np.max(y2_i)
            if len(y1_i) == 1:
                out1[2, i] = np.max(y1_i)
                out2[2, i] = np.max(y2_i)
            else:
                out1[1, i] = np.max(y1_i)
                out2[1, i] = np.max(y2_i)

    plots[2].line(vec, out1[0, :], line_width=2, color=palette[kk + 1])
    plots[2].scatter(vec, out1[1, :], line_width=1, size=6, color=palette[kk + 1], legend_label=f'{gi/g} γ')
    plots[2].scatter(vec, out1[2, :], line_width=1, size=6, color=palette[kk + 1], fill_color='white')

    plots[3].line(vec, out2[0, :], line_width=2, color=palette[kk + 1])
    plots[3].scatter(vec, out2[1, :], line_width=1, size=6, color=palette[kk + 1])
    plots[3].scatter(vec, out2[2, :], line_width=1, size=6, color=palette[kk + 1], fill_color='white')

plots[2].yaxis.axis_label = 'y1 [uM]'
plots[3].yaxis.axis_label = 'y2 [uM]'
#plots[2].y_range = Range1d(2.0, 2.51)
#plots[3].y_range = Range1d(0, 2.5)
plots[2].legend.background_fill_alpha = 0.0
plots[2].legend.location = 'top_right'
plots[2].title.text = 'Positive feedback'

# Visualization
bokeh.plotting.show(bokeh.layouts.gridplot([[plots[0], plots[1], None], [plots[2], plots[3], None]], toolbar_location='above'))

<ipython-input-3-0b44bbe636e5>:65: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m)
<ipython-input-3-0b44bbe636e5>:31: RuntimeWarning: divide by zero encountered in divide
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d
<ipython-input-3-0b44bbe636e5>:31: RuntimeWarning: invalid value encountered in power
  y2_1 = (a*K**m/(d*y - b*u1) - K**m)**(1/m) # (1) Eq y1(y2) (hill + b*u1) / d


In [7]:
#@title How much can we deviate from the adaptive metric? - Part 2
a = 2.2  # production rate [uM/h]
K = 1    # Kappa [uM]
m = 3    # cooperativity
d = 1    # delta [1/h]
k = 1    # production rate [1/h]
gv = (10, 100, 1000)  # sequestration rates [1/h/uM]
th = d   # production rate [uM/h]
xi = 1   # production rate [uM/h]
b = 2    # gain [1/h]
Omega = 100  # or 600

time_points = np.linspace(0, 20, 101)
population_0 = np.zeros(5, dtype=int)

# Parameters to analyze
args1 = (a, K, m, d, b*0, k, xi, th, gv[0], Omega)
args2 = (a, K, m, d, b, k, xi*1.15, th, gv[0], Omega)
args3 = (a, K, m, d, b, k, xi*1.2, th, gv[0], Omega)
args4 = (a, K, m, d, b, k, xi*0.85, th, gv[0], Omega)
args5 = (a, K, m, d, b, k, xi*0.8, th, gv[0], Omega)

# Solving with Gillespie algorithm
samples1 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args1, n_threads=4, progress_bar=False)
samples2 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args2, n_threads=4, progress_bar=False)
samples3 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args3, n_threads=4, progress_bar=False)
samples4 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args4, n_threads=4, progress_bar=False)
samples5 = biocircuits.gillespie_ssa(simple_propensity_PF, simple_update, population_0, time_points, size=250, args=args5, n_threads=4, progress_bar=False)

# Set up plots for probability analysis
fig_size = [320, 225]  # to visualize
x_range = (-300, 300)
y_range = (0, 0.02)

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                                       x_range=x_range, y_range=y_range,
                                       x_axis_label = 'y1 - y2 [# molecules]', y_axis_label = 'p'))
    plots[i].axis.major_label_text_font_size = "10pt"

# β = 0 as a reference
bin0 = np.arange(-3*Omega, 3*Omega+6, 8)  # 8
hist, bin_edges = np.histogram(samples1[:, -1, 0] - samples1[:, -1, 1], bin0, density=True)

for i in range(6):
    if i != 3:
        plots[i].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="grey", line_color="grey", fill_alpha=0.2, line_alpha=0.)
        plots[i].step(bin_edges[:-1], hist, line_width=1, mode="after", color="grey")

# Nulcliness for β = 0 as a reference
y1_1 = np.linspace(0,3,300)
y2_2 = np.linspace(0,3,300)
y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b*0, k, xi, th, gv[0])
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[3].line(y1_1, y2_1, line_width=2, color='black', alpha=0.2)
plots[3].line(y1_2, y2_2, line_width=2, color='black', alpha=0.2)
plots[3].scatter(y1_i, y2_i, line_width=1, color='gray', size=7, legend_label='control')
plots[3].xaxis.axis_label = 'y1 [uM]'
plots[3].yaxis.axis_label = 'y2 [uM]'
plots[3].x_range = Range1d(0, 3.1)
plots[3].y_range = Range1d(0, 3.1)

# Probability density function for a 15% variation
hist, bin_edges = np.histogram(samples2[:, -1, 0] - samples2[:, -1, 1], bin0, density=True)

vec_xi = [1.15, 0.85]  # for a 15% variation

plots[1].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="orange", line_color="orange", fill_alpha=0.2, line_alpha=0.)
plots[1].step(bin_edges[:-1], hist, line_width=1, mode="after", color="orange")
plots[1].title.text = f'r = {vec_xi[0]*d/(k*th):.2f}'

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, vec_xi[0], th, gv[0])
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[3].scatter(y1_i, y2_i, line_width=1, color='orange', size=7, legend_label='+15%')

# Probability density function for a 20% variation
hist, bin_edges = np.histogram(samples3[:, -1, 0] - samples3[:, -1, 1], bin0, density=True)

plots[2].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="blue", line_color="blue", fill_alpha=0.2, line_alpha=0.)
plots[2].step(bin_edges[:-1], hist, line_width=1, mode="after", color="blue")
plots[2].title.text = f'r = {vec_xi[1]*d/(k*th):.2f}'

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, vec_xi[1], th, gv[0])
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[3].scatter(y1_i, y2_i, line_width=1, color='blue', size=7, legend_label=f'+20%')

# Probability density function for a 40% variation
hist, bin_edges = np.histogram(samples4[:, -1, 0] - samples4[:, -1, 1], bin0, density=True)

vec_xi = [1.2, 0.8]  # for a 40% variation

plots[4].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="purple", line_color="purple", fill_alpha=0.2, line_alpha=0.)
plots[4].step(bin_edges[:-1], hist, line_width=1, mode="after", color="purple")
plots[4].title.text = f'r = {vec_xi[0]*d/(k*th):.2f}'

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, vec_xi[0], th, gv[0])
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[3].scatter(y1_i, y2_i, line_width=1, color='purple', size=7, legend_label=f'-15%')

# Probability density function for a 60% variation
hist, bin_edges = np.histogram(samples5[:, -1, 0] - samples5[:, -1, 1], bin0, density=True)

plots[5].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:], fill_color="green", line_color="green", fill_alpha=0.2, line_alpha=0.)
plots[5].step(bin_edges[:-1], hist, line_width=1, mode="after", color="green")
plots[5].title.text = f'r = {vec_xi[1]*d/(k*th):.2f}'

y2_1, y1_2 = PF_null(y1_1, y2_2, a, K, m, d, b, k, vec_xi[1], th, gv[0])
y1_i, y2_i = intersection(y1_1, y2_1, y1_2, y2_2)

plots[3].scatter(y1_i, y2_i, line_width=1, color='green', size=7, legend_label=f'-20%')

# Generating a grid plot
layout = bokeh.layouts.gridplot([[plots[0], plots[1], plots[2]], [plots[3], plots[4], plots[5]]])

# Visualization
bokeh.plotting.show(layout)